使用原生pytorch实现最原生的linear GNN模型，论文为2009年的[《The graph neural network model》](https://persagen.com/files/misc/scarselli2009graph.pdf)的GNN,
代码部分大幅借鉴了[Github用户LYuhang](https://github.com/LYuhang/GNN_Review/blob/master/PyG%E5%92%8CPytorch%E5%AE%9E%E7%8E%B0GNN%E6%A8%A1%E5%9E%8B/GNN_Implement_with_Pytorch.ipynb),建议大家以他的讲解为主，我的作为一些细节的补充

In [2]:
import os
import numpy as np
#Tqdm 是一个快速，可扩展的Python进度条，可以在 Python 长循环中添加一个进度提示信息，用户只需要封装任意的迭代器 tqdm(iterator)
from tqdm import tqdm
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F

device = 'cpu'
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


使用的数据集为[cora数据集](https://linqs.soe.ucsc.edu/data)(该网站还有其它关于图神经网络的数据集),该数据集由许多机器学习领域的paper构成，这些paper被分为7个类别，在该数据集中，一篇论文至少与该数据集中任一其它论文有引用或被引用关系，共2708篇论文

总共包含两个文件：  
1.`cora.content`文件包含对paper的内容描述，格式为$$ \text{<paper_id> <word_attributes> <class_label>} $$其中：  
&emsp;`<paper_id>`是paper的标识符，每一篇paper对应一个标识符。  
&emsp;`<word_attributes>`是词汇特征，为0或1，表示对应词汇是否存在。  
&emsp;`<class_label>`是该文档所述的类别。  
  
2.`cora.cites`包含了数据集的引用图，格式为$$ \text{<ID of cited paper> <ID of citing paper>} $$其中：  
&emsp;`<ID of cited paper>`是被引用的paper标识符。  
&emsp;`<ID of citing paper>`是引用的paper标识符。 

In [60]:
'''
node_num, feat_dim, stat_dim, num_class, T
feat_Matrix, X_Node, X_Neis, dg_list, label_egde
'''
#数据处理
content_path = "./cora/cora.content"
cite_path = "./cora/cora.cites"

#读取文本内容
with open(content_path, "r") as fp:
    contents = fp.readlines()
with open(cite_path, "r") as fp:
    cites = fp.readlines()
    
contents = np.array([np.array(s.strip().split("\t")) for s in contents])
paper_list, feat_list, label_list = np.split(contents, [1,-1], axis=1)
paper_list, label_list = np.squeeze(paper_list), np.squeeze(label_list)

# paper -> index dict
#print("paper_list",sorted(paper_list))
paper_dict = dict([(key, val) for val, key in enumerate(paper_list)])

# lable -> index dict
labels = list(set(label_list))
label_dict = dict([(key, val) for val, key in enumerate(labels)])

# edge_index
cites = [i.strip().split("\t") for i in cites]
#print(cites)
#下面这几句代码不一样
#print(paper_dict)
cites = np.array([[paper_dict[i[0]], paper_dict[i[1]]] for i in cites], dtype = np.int64)
#print(cites[1:7])
cites = np.concatenate((cites, cites[:, ::-1]), axis=0) 
#print(cites[1:7])
#这句也不一样
#print(cites[:,0])
degree_list=np.zeros(len(paper_list), dtype = np.int32)
for i in cites:
    degree_list[i[0]] += 1
#_, degree_list = np.unique(cites[:,0],return_counts=True)
#print(degree_list)

#input
node_num = len(paper_list)
feat_dim = feat_list.shape[1]
stat_dim = 32
num_class = len(labels)
edge_dim = 2
T = 2
feat_Matrix = torch.Tensor(feat_list.astype(np.float32))
X_Node, X_Neis = np.split(cites, 2, axis=1)
X_Node, X_Neis = torch.tensor(np.squeeze(X_Node)), \
                 torch.tensor(np.squeeze(X_Neis))
#print(X_Node)
#print(degree_list[163])
dg_list = torch.tensor(degree_list[X_Node])
label_list = np.array([label_dict[i] for i in label_list])
label_list = torch.tensor(label_list, dtype = torch.long)
label_edge = torch.tensor([[0., 1.]] * (len(dg_list)//2)+[[1., 0.]] * (len(dg_list)//2))  #这个ont-hot向量分别表示引用边和被引用边
#print(label_edge)

In [61]:
print("Number of node : ", node_num)
print("Number of edges : ", cites.shape[0])
print("Number of classes : ", num_class)
print("Dimension of node features : ", feat_dim)
print("Dimension of node state : ", stat_dim)
print("Shape of feat_Matrix : ", feat_Matrix.shape)
print("Shape of X_Node : ", X_Node.shape)
print("Shape of X_Neis : ", X_Neis.shape)
print("Length of dg_list : ", len(dg_list))
print("Length of label_edge : ", len(label_edge))

Number of node :  2708
Number of edges :  10858
Number of classes :  7
Dimension of node features :  1433
Dimension of node state :  32
Shape of feat_Matrix :  torch.Size([2708, 1433])
Shape of X_Node :  torch.Size([10858])
Shape of X_Neis :  torch.Size([10858])
Length of dg_list :  10858
Length of label_edge :  10858


![avatar](equation1.jpg)
![avatar](equation2.jpg)
![avatar](equation3.jpg)
其中，关于各参数的解释见论文第10页,以下附上截图：  
其中s是state的维度
![avatar](linearGNN.png)

In [62]:
'''
下面Xi函数是生成论文中A矩阵方程式中的Xi矩阵(就是符号很奇怪的那个矩阵）。
然后我的phi函数是直接把ln,lu拼接起来，
l(n,u)没有（其实这里应该加上更好，引用边和被引用边显然应该用不同的label）
Initialization :
Input :
    ln : (int)特征向量维度
    le : (int)边的特征向量维度
    s : (int)状态向量维度
Forward : 
N为节点数
Input :
    input : (Tensor)节点对(i,j)的特征向量拼接起来，shape为(N，2*ln+le)
Output :
    out : (Tensor)Xi矩阵，shape为(N, s, s)
'''
class Xi(nn.Module):
    def __init__(self, ln, s, le):
        super(Xi, self).__init__()
        self.ln = ln
        self.s = s
        
        self.linear = nn.Linear(2 * ln + le, s ** 2)
    
    def forward(self, input):
        #在jupyter中用F.tanh会有警告，它会让你用torch.tanh
        #它的警告是F.tanh is deprecated,但是我网上都没有查到这个
        #不过反正torch.tanh是一样的，那就不用F.tanh了
        output = torch.tanh(self.linear(input))
        return output.view(-1, self.s, self.s)

In [63]:
#测试正确与否：
model1 = Xi(5,3 ,2)
input1 = torch.randn(4,12)
print(model1(input1))

tensor([[[ 0.6787, -0.1234,  0.0259],
         [-0.4561, -0.9040, -0.1219],
         [ 0.2899,  0.2537, -0.4306]],

        [[-0.4246,  0.2783,  0.0912],
         [ 0.7354, -0.2594, -0.4767],
         [-0.5070,  0.0617, -0.4514]],

        [[ 0.6619, -0.2249,  0.4534],
         [-0.8988,  0.3727, -0.4289],
         [ 0.4077, -0.1982, -0.8739]],

        [[ 0.2755,  0.4321,  0.0447],
         [ 0.3188, -0.3134, -0.7724],
         [ 0.0145, -0.4467, -0.6117]]], grad_fn=<ViewBackward>)


In [64]:
'''
实现论文中rou矩阵，并生成偏置项b
Initialization :
Input :
    ln : (int)特征向量维度
    s : (int)状态向量维度
Forward :
Input :
    input : (Tensor)节点的特征向量矩阵，shape(N, ln)
Output :
    out : (Tensor)偏置矩阵，shape(N, s)
'''
class Rou(nn.Module):
    def __init__(self, ln, s):
        super(Rou, self).__init__()
        self.linear = nn.Linear(ln,s)
    
    def forward(self, input):
        return torch.tanh(self.linear(input))

In [65]:
model1 = Rou(5,3)
input1 = torch.randn(4,5)
print(model1(input1))

tensor([[ 0.3827,  0.2693, -0.4215],
        [ 0.4116,  0.5820,  0.0045],
        [-0.3160,  0.1672,  0.3444],
        [ 0.2149,  0.5229, -0.0913]], grad_fn=<TanhBackward>)


In [66]:
'''
实现Hw函数，就是论文中的公式12,其中|ne[u]|的意思是ne[u]邻居的个数
Initialize :
Input :
    ln : (int)节点特征向量维度
    s : (int)节点状态向量维度
    le : (int)边的特征向量维度
    mu : (int)设定的压缩映射的压缩系数
Forward :
Input :
    X : (Tensor)每一行为一条边的两个节点特征向量外加这条边的特征向量连接起来得到的向量，shape为(N, 2*ln + le)
    H : (Tensor)与X每行对应u=one of ne[n](即x中的第二个节点,这里默认边是第二个点连到第一个，即2是1的ne,但1不是2的ne)的状态向量
    dg_list: (Tensor)与X每行对应u=one of ne[n]的度数向量ne[u]
Output :
    out : (Tensor)Hw函数的输出
'''
class Hw(nn.Module):
    def __init__(self, ln, s, le, mu=0.9):
        super(Hw, self).__init__()
        self.ln = ln
        self.s = s
        self.le = le
        self.mu = mu
        
        self.Xi = Xi(ln, s, le)
        self.Rou = Rou(ln, s)
    
    '''
    A: N * s * s
    b: N * s
    '''
    def forward(self, X, H, dg_list):
        A = (self.Xi(X) * self.mu / self.s)/dg_list.view(-1, 1, 1)
        b = self.Rou(X[:,0:self.ln])#chunk函数：分割tensor，(n,ln)->(n,s)
        #下面必须这样先unsqueeze再squeeze,可以参考matmul规则
        #matmul高维矩阵相乘：自动在前补1*，使维数相同，然后广播，使高于最后两维的size相同
        #然后依次对里面进行二维的矩阵乘法
        #如(j*1*n*p)(k*p*m)->(j*k*n*p)(j*k*p*m)->(j*k*n*m) 
        output = torch.squeeze(torch.matmul(A, torch.unsqueeze(H,2)),-1) + b
        return output

In [67]:
model1 = Hw(5,3,2)
X1 = torch.randn(4,12)
H1 = torch.randn(4,3)
dg_list1 = torch.tensor([2,3,4,5])
print(model1(X1, H1, dg_list1))

tensor([[-0.0025, -0.1701, -0.1836],
        [ 0.6183,  0.1260, -0.0162],
        [ 0.0338, -0.6600,  0.1344],
        [ 0.4552, -0.1040, -0.1249]], grad_fn=<AddBackward0>)


In [68]:
'''
实现上面论文中的方程（3），将前面使用的hw函数得到的信息相加了，
并更新每一个节点的状态向量
Initialize :
Input :
    node_num : (int)节点的数量
Forward :
Input :
    H : (Tensor)Hw的输出，shape为(E, s)
    X_node : (Tensor)H每一行对应要更新的节点的索引(E)
Output :
    out :(Tensor)新的节点状态向量，shape为(N, s), N为节点个数
'''
class AggrSum(nn.Module):
    def __init__(self, node_num):
        super(AggrSum, self).__init__()
        self.N = node_num
        
    def forward(self, H, X_node):
        #H : (E, s) -> (N,s)
        #感觉写出矩阵乘法有一点点浪费时间和空间
        #但是for循环寻址也挺慢的,方便来看还是写矩阵乘法吧
        mask = torch.stack([X_node] * self.N, 0)
        mask = mask.float() - torch.unsqueeze(torch.arange(0,self.N).float(), 1)
        mask = (mask == 0).float()
        # (V, N) * (N, s) -> (V, s)
        return torch.mm(mask, H)

In [69]:
model1 = Hw(5,3,2)
X1 = torch.randn(4,12)
H1 = torch.randn(4,3)
dg_list1 = torch.tensor([2,3,4,5])
print(model1(X1, H1, dg_list1))
model2 = AggrSum(2)
print(model2(model1(X1, H1, dg_list1), torch.tensor([0,1,1,0])))

tensor([[-0.1437,  0.5951,  0.3575],
        [-0.8037,  0.6757,  0.4867],
        [-0.3223,  0.2060,  0.3875],
        [ 0.2539, -0.9173,  0.1773]], grad_fn=<AddBackward0>)
tensor([[ 0.1101, -0.3221,  0.5348],
        [-1.1260,  0.8818,  0.8742]], grad_fn=<MmBackward>)


In [70]:
'''
实现Linear GNN模型，循环迭代计算T次，
达到不动点之后，使用线性函数得到输出，进行分类
Initialize : 
Input :
    node_num : (int)节点个数
    feat_dim : (int)节点特征向量维度
    stat_dim : (int)节点状态向量维度
    edge_dim : (int)边特征向量维度
    T : (int)迭代计算的次数
Forward :
Input :
    feat_Matrix : (Tensor)节点的特征矩阵，shape为（N，ln)
    X_Node : (Tensor)每条边的提供更新的节点的索引，例如i->j,i就是提供更新的，（N）
    X_Neis : (Tensor)每条边被更新的节点的索引，（N）
    dg_list : (Tensor)与X_Node对应节点的度列表，shape为（N）
    label_edge : (Tensor）特征向量
    out : (Tensor)每个节点的类别概率，shape为（V，num_class)
'''
class OriLinearGNN(nn.Module):
    def __init__(self, node_num, feat_dim, stat_dim, num_class, edge_dim,T):
        super(OriLinearGNN, self).__init__()
        self.embed_dim = feat_dim
        self.stat_dim = stat_dim
        self.edge_dim = edge_dim
        self.T = T
        
        self.out_layer = nn.Linear(stat_dim, num_class)
        #这里天坑，由于不需要学习的我一向喜欢直接用F.而不是类
        #结果之前一直准确率不对，才发现如果用的是类在预测时会自动关闭的，而函数则不会
        self.dropout = nn.Dropout()
        
        self.Hw = Hw(feat_dim, stat_dim, edge_dim)
        self.Aggr = AggrSum(node_num)
        
    def forward(self, feat_Matrix, X_Node, X_Neis, dg_list, label_edge):
        #这里是取出要用到的特征向量
        node_embeds = torch.index_select(feat_Matrix, 0, X_Node)
        neis_embeds = torch.index_select(feat_Matrix, 0, X_Neis)
        X = torch.cat((node_embeds, neis_embeds, label_edge), 1) #N, 2*ln + le
        #H是其中的状态向量
        H = torch.zeros((feat_Matrix.shape[0], self.stat_dim), dtype = torch.float32).to(device)
        for t in range(self.T):
            #(V, s) -> (N, s)
            #这里是取出要用到的状态向量
            H = torch.index_select(H, 0, X_Neis)
            #(N, s) -> (N, s)
            #这里是得到能拿来更新状态向量的矩阵
            H = self.Hw(X, H, dg_list)
            #(N, s) -> (V, s)
            #这里是更新状态向量
            H = self.Aggr(H, X_Node)
        output = F.log_softmax(self.dropout(self.out_layer(H)),dim = -1)
        return output

In [73]:
#split dataset
train_mask = torch.zeros(node_num, dtype = torch.bool)
train_mask[:node_num - 1000] = 1               #1700左右training

val_mask = None                                
test_mask = torch.zeros(node_num, dtype = torch.bool)
test_mask[node_num - 500:] = 1                 # 500test

model = OriLinearGNN(node_num, feat_dim, stat_dim, num_class, edge_dim, T).to(device)

#Adam是一种算法，可以百度了解
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01, weight_decay = 1e-3)
feat_Matrix = feat_Matrix.to(device)
X_Node = X_Node.to(device)
X_Neis = X_Neis.to(device)

for epoch in range(50):
    model.train()
    optimizer.zero_grad()

    out = model(feat_Matrix, X_Node, X_Neis, dg_list, label_edge)
    
    loss = F.nll_loss(out[train_mask], label_list[train_mask])
    _, pred = out.max(dim=1)
    
    correct = float(pred[train_mask].eq(label_list[train_mask]).sum().item())
    acc = correct / train_mask.sum().item()
    print('[Epoch {}/200] Loss {:.4f}, train acc {:.4f}'.format(epoch, loss.cpu().detach().data.item(), acc))
    
    loss.backward()
    optimizer.step()
    
    if (epoch + 1) % 10 == 0:
        model.eval()
        _, pred = model(feat_Matrix, X_Node, X_Neis, dg_list, label_edge).max(dim = 1)
        correct = float(pred[test_mask].eq(label_list[test_mask]).sum().item())
        acc = correct / test_mask.sum().item()
        print('Accuracy: {:.4f}'.format(acc))

[Epoch 0/200] Loss 1.9906, train acc 0.1399
[Epoch 1/200] Loss 1.5634, train acc 0.4251
[Epoch 2/200] Loss 1.3289, train acc 0.5281
[Epoch 3/200] Loss 1.1532, train acc 0.5738
[Epoch 4/200] Loss 1.0439, train acc 0.5937
[Epoch 5/200] Loss 1.0078, train acc 0.5896
[Epoch 6/200] Loss 0.9554, train acc 0.5937
[Epoch 7/200] Loss 0.8937, train acc 0.5972
[Epoch 8/200] Loss 0.8687, train acc 0.6107
[Epoch 9/200] Loss 0.8346, train acc 0.6001
Accuracy: 0.7720
[Epoch 10/200] Loss 0.7979, train acc 0.6224
[Epoch 11/200] Loss 0.8005, train acc 0.6189
[Epoch 12/200] Loss 0.8298, train acc 0.5978
[Epoch 13/200] Loss 0.7673, train acc 0.6276
[Epoch 14/200] Loss 0.7700, train acc 0.6388
[Epoch 15/200] Loss 0.7755, train acc 0.6171
[Epoch 16/200] Loss 0.7395, train acc 0.6370
[Epoch 17/200] Loss 0.7758, train acc 0.6183
[Epoch 18/200] Loss 0.7412, train acc 0.6265
[Epoch 19/200] Loss 0.7543, train acc 0.6253
Accuracy: 0.7780
[Epoch 20/200] Loss 0.7376, train acc 0.6335
[Epoch 21/200] Loss 0.7483, tra